In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
! ls


gdrive	sample_data


In [ ]:
! git clone https://github.com/echen4628/bert_for_qa.git


Cloning into 'bert_for_qa'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 17 (delta 5), reused 16 (delta 4), pack-reused 0
Unpacking objects: 100% (17/17), 5.14 KiB | 1.28 MiB/s, done.


In [ ]:
%cd bert_for_qa

/content/bert_for_qa


In [ ]:
! git pull

Already up to date.


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import default_data_collator, AutoModelForQuestionAnswering, AutoTokenizer
from dataset import *
from torch.optim import AdamW
from accelerate import Accelerator
from transformers import get_scheduler
from huggingface_hub import Repository, get_full_repo_name, notebook_login
from tqdm.auto import tqdm
import evaluate
import collections
from training import *
import functools

In [ ]:
  # configuring data
  squad = datasets.load_dataset("squad")
  # train_small = raw_datasets["train"][:5]
  # validation_small = raw_datasets["validation"][:5]

  # train_small = raw_datasets["train"][:5]
  # validation_small = raw_datasets["validation"][:5]
  # small_dataset = {'train':Dataset.from_dict(train_small),
  #     'validation':Dataset.from_dict(validation_small)}

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
rng = np.random.default_rng(42)
def generate_datasubset(dataset, percentage, rng):
  unique_train_title = np.unique(dataset["train"]["title"])
  rng.shuffle(unique_train_title)
  train_titles = unique_train_title[:int(len(unique_train_title)*percentage)]

  unique_val_title = np.unique(dataset["validation"]["title"])
  rng.shuffle(unique_val_title)
  val_titles = unique_val_title[:int(len(unique_val_title)*percentage)]
  

  new_dataset = dataset.filter(lambda example: (example['title'] in train_titles) or (example['title'] in val_titles) and example["answers"]["answer_start"] != [])
  return new_dataset


# unique_titles = np.unique(squad2["train"]["title"])
# rng.shuffle(unique_titles)
# unique_titles[:44]

In [ ]:
small_dataset = generate_datasubset(squad, 0.05, rng)

Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
np.unique(small_dataset["train"]["title"])

array(['2008_Sichuan_earthquake', 'Baptists', 'Beer', 'Bird_migration',
       'British_Isles', 'Catalan_language', 'Christian', 'Electric_motor',
       'England_national_football_team', 'Eritrea', 'Frédéric_Chopin',
       'Hellenistic_period', 'LaserDisc', 'Late_Middle_Ages',
       'Molotov%E2%80%93Ribbentrop_Pact', 'New_Haven,_Connecticut',
       'Race_(human_categorization)', 'Sanskrit', 'Space_Race',
       'Tucson,_Arizona', 'Umayyad_Caliphate',
       'Washington_University_in_St._Louis'], dtype='<U34')

In [ ]:
len(np.unique(list(map(lambda x: str(x["text"]), small_dataset["train"]["answers"]))))

4658

In [ ]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

max_length = 384
stride = 128

train_dataset = squad["train"].map(
                lambda x: preprocess_training_examples(tokenizer, x),
                batched=True,
                remove_columns=squad["train"].column_names,)

validation_dataset = squad["validation"].map(
                    lambda x: preprocess_validation_examples(tokenizer, x),
                    batched=True,
                    remove_columns=squad["validation"].column_names,)

train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
# preparing for training
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)


model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)


optimizer = AdamW(model.parameters(), lr=2e-5)

accelerator = Accelerator(mixed_precision='fp16')
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader)

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
metric = evaluate.load("squad")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [ ]:
notebook_login()
model_name = "bert-finetuned-squad-full"
repo_name = get_full_repo_name(model_name)
print(f"The huggingface repository name is : {repo_name}")
output_dir = "bert-finetuned-squad-full"
repo = Repository(output_dir, clone_from=repo_name)


Cloning https://huggingface.co/echen4628/bert-finetuned-squad-full into local empty directory.


The huggingface repository name is : echen4628/bert-finetuned-squad-full


In [ ]:

train_model(num_training_steps, num_train_epochs, model,tokenizer, squad, train_dataloader, validation_dataset, eval_dataloader, 
            accelerator,optimizer, lr_scheduler, output_dir, metric, repo)



  0%|          | 0/33276 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/1353 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 0: {'exact_match': 79.92431409649953, 'f1': 87.350554544426}
Evaluation!


...

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 1: {'exact_match': 81.0406811731315, 'f1': 88.4081536037188}
Evaluation!


  0%|          | 0/1353 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 2: {'exact_match': 80.82308420056765, 'f1': 88.44673351768431}


In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "bert-finetuned-squad-full"
question_answerer = pipeline("question-answering", model=model_checkpoint)

context = """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "Which deep learning libraries back 🤗 Transformers?"
question_answerer(question=question, context=context)

{'score': 0.9816339015960693,
 'start': 78,
 'end': 105,
 'answer': 'Jax, PyTorch and TensorFlow'}

In [ ]:
context = """
Sacramento has a population of people, which is much less than the biggest cities in the state. The five cities larger by population than the state capital, Sacramento, are Los Angeles, San Diego, San Jose, San Francisco, and Fresno.

"""
question = "Is Sacramento the biggest city"
question_answerer(question=question, context=context)

{'score': 0.30625343322753906,
 'start': 49,
 'end': 95,
 'answer': 'much less than the biggest cities in the state'}

In [ ]:
context = """
Different sources provide the authority for tax rules and procedures. Here are some sources that can be searched online for free.

Internal Revenue Code
The Constitution gives Congress the power to tax. Congress typically enacts Federal tax law in the Internal Revenue Code of 1986 (IRC).

The sections of the IRC can be found in Title 26 of the United States Code (26 USC). An electronic version of the current United States Code is made available to the public by Congress.

Browse "Title 26—Internal Revenue Code" to see the table of contents for the IRC.
"Jump To" to a specific section of Title 26 to find the text for that IRC provision. For example, you can "Jump To" Title 26 Section 24 to find the provision for the child tax credit in the IRC.
Use the Advanced Search feature to search only in Title 26 for a specific term. For example, you can search for "child tax credit" in Title 26 to find section 24 of the IRC, shown as 26 USC 24.
Caution: Before relying on any IRC section retrieved from this or any other website, check to see whether the provision that's displayed shows laws that became effective after the tax year you're researching. It's also possible that updates from a recent law haven't yet been reflected in the provision. Additionally, sometimes Congress enacts laws that are not part of the IRC but nonetheless impact Federal tax law. Historical versions of the United States Code (back to 1994) are available electronically on GovInfo, a website from the U.S. Government Publishing Office (GPO).

The IRC is complex, and its sections must be read in the context of the entire Code, the Treasury Regulations, and the court decisions that interpret it.

Since shortly after the federal income tax was enacted in 1913, some individuals and groups have encouraged others not to comply with the tax laws. There have been unsuccessful challenges about the applicability of tax laws using a variety of arguments. Despite the courts having consistently rejected these arguments, their promoters continue to expound them, even incurring penalties for bringing frivolous cases into court or for filing frivolous tax returns. They often present their arguments in a pseudo-legal format, luring unsuspecting people into participating in their schemes to evade taxes Do not be misled by the false interpretations of the IRC promoted by the purveyors of anti-tax law evasion schemes.

Treasury (Tax) Regulations
Treasury Regulations—commonly referred to as Federal tax regulations—provide the official interpretation of the IRC by the U.S. Department of the Treasury and give directions to taxpayers on how to comply with the IRC's requirements.

Treasury Regulation sections can be found in Title 26 of the Code of Federal Regulations (26 CFR). An electronic version of the current Code of Federal Regulations is made available to the public by the National Archives and Records Administration (NARA) and the GPO.

Browse "Title 26 :: Internal Revenue" to see the table of contents for the current Treasury Regulations.
"Go" to a specific section of Title 26 to find the current text for that Treasury Regulation. For example, you can "Go" to 26 CFR 1.61-1 to find regulations defining gross income.
Search for a term only in Title 26. For example, you can search on "gross income" in Title 26 to find the current text for 26 CFR 1.61-1.
Historical versions of the Code of Federal Regulations (back to 1996) are available on GovInfo.
"""
question = "What is gross income"
question_answerer(question=question, context=context)

{'score': 0.005000152159482241,
 'start': 3322,
 'end': 3356,
 'answer': 'the current text for 26 CFR 1.61-1'}